## Doublets Examples

One for inhibitory, the other for excitatory

In [ ]:
from pathlib import Path

import sys
import os
sys.path.append(os.path.abspath("../src"))

import importlib
import utils
import plot

importlib.reload(utils)
importlib.reload(plot)

from utils import load_merfish_signal_df, load_banksy_result, load_merfish_data, load_boundaries_data
from plot import CM

import anndata as ad
import numpy as np
import polars as pl
import matplotlib.pyplot as plt

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.patches import Rectangle
from typing import Any

from matplotlib.colors import LinearSegmentedColormap

BIH_CMAP = LinearSegmentedColormap.from_list(
    "BIH",
    [
        "#430541",
        "mediumvioletred",
        "violet",
        "powderblue",
        "powderblue",
        "white",
        "white",
    ][::-1],
)

SCALEBAR_PARAMS: dict[str, Any] = {
    "dx": 1,
    "units": "um",
    "length_fraction": 0.1,
    "location": "lower right",
    "pad": 0.1,
    "frameon": False
}

def _plot_scalebar(ax, dx=1, units="um", fontsize=5, color="black", box_alpha=0, **kwargs):
    scalebar = ScaleBar(
        dx,
        units=units,
        scale_loc="top", 
        sep=1,
        **kwargs
    )
    scalebar.set_font_properties({"size": fontsize})
    scalebar.linewidth = 0.3
    scalebar.box_alpha = box_alpha
    scalebar.color = color
    ax.add_artist(scalebar)

### data

#### Signals in the Tissue Section

In [ ]:
MERFISH_data_folder_path = Path("../data/mouse_hypothalamus/MERFISH/")

In [ ]:
signal_coordinate_df, coordinate_x_m, coordinate_y_m = load_merfish_signal_df(MERFISH_data_folder_path/"merfish_barcodes_example.csv")

#### Results of BANKSY

In [ ]:
banksy_folder_path = Path("../data/banksy_results/")

In [ ]:
banksy_result = load_banksy_result(banksy_folder_path/"banksy_cluster.txt", coordinate_x_m, coordinate_y_m)

#### Segmentation Dataset

In [ ]:
merfish_data = load_merfish_data(MERFISH_data_folder_path / "merfish_all_cells.csv", 
                                 banksy_result, coordinate_x_m, coordinate_y_m, 
                                 animal_id=1, bregma_value=-0.24)

#### Cell boundaries

In [ ]:
boundaries_df = load_boundaries_data(MERFISH_data_folder_path/'cellboundaries_example_animal.csv', 
                                     merfish_data, coordinate_x_m, coordinate_y_m)

### Doublets

In [ ]:
ovrlpy_result_folder = Path("../data/results/VSI/")

signal_integrity = np.loadtxt(ovrlpy_result_folder/"SignalIntegrity.txt")
signal_strength = np.loadtxt(ovrlpy_result_folder/"SignalStrength.txt")
pseudocells = ad.read_h5ad(ovrlpy_result_folder/"pseudocells.h5ad")
doublets = pl.read_csv(ovrlpy_result_folder/"doublets_df.csv")

In [ ]:
windowsize = 60
window_size = windowsize / 2
signal_threshold=3.0
roi_scatter_kwargs = dict(marker=".", alpha=0.8, s=11)

#### UMAP

In [ ]:
embedding = pseudocells.obsm["2D_UMAP"]
RGB = pseudocells.obsm["RGB"]

In [ ]:
fig, ax = plt.subplots(figsize=(8*CM, 8*CM), dpi=600)
ax.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=RGB,
    alpha=0.5,
    marker=".",
    edgecolors='none',
    s=5,
)
ax.set(aspect='equal', adjustable='box')

ax.set_xlim(-8,17)
ax.set_ylim(-8,17)

for spine in ax.spines.values():
        spine.set_linewidth(0.3)

ax.set_xticks([])
ax.set_yticks([])
plt.title("UMAP", fontsize=7)
plt.tight_layout()
plt.savefig("./UMAP.pdf")
plt.show()


#### Cell Map

In [ ]:
spatial = pseudocells.obsm["spatial"]

In [ ]:
fig, ax = plt.subplots(figsize=(8*CM, 8*CM), dpi=600)
ax.scatter(
    spatial[:, 0],
    spatial[:, 1],
    c=RGB,
    alpha=0.8,
    marker=".",
    edgecolors='none',
    s=5,
)
ax.set(aspect="equal")

x_center, y_center = doublets["x", "y"].row(49)
x0 = x_center - window_size
y0 = y_center - window_size
square = Rectangle(
    (x0, y0),
    width=windowsize,
    height=windowsize,
    linewidth=0.4,
    fill=False,
    edgecolor="k",
)
ax.add_patch(square)
ax.text(
            x0 + windowsize / 2, y0 + windowsize + 10,
            "ROI 1",
            color="k",
            fontsize=5,
            fontweight='light',
            ha='center'
        )

x_center, y_center = doublets["x", "y"].row(158)
x0 = x_center - window_size
y0 = y_center - window_size
square = Rectangle(
    (x0, y0),
    width=windowsize,
    height=windowsize,
    linewidth=0.4,
    fill=False,
    edgecolor="k",
)
ax.add_patch(square)
ax.text(
            x0 + windowsize / 2, y0 + windowsize + 10,
            "ROI 2",
            color="k",
            fontsize=5,
            fontweight='light',
            ha='center'
        )

_plot_scalebar(ax, dx=1, units="um", location="lower left", length_fraction=0.25, fontsize=5, box_alpha=1)

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
        spine.set_linewidth(0.3)
ax.set_xlim(-30,1830)
ax.set_ylim(-30,1830)
ax.set_aspect('equal')
plt.title("Cell Type Map", fontsize=7)
plt.tight_layout()
plt.show()

#### Doublet1: Inhibitory

doublet_to_show = 49  
x,y = 292, 924

In [ ]:
ROI_in = pl.read_parquet(ovrlpy_result_folder/"roi_transcripts_In.parquet")

doublet_to_show = 49
x, y = doublets["x", "y"].row(doublet_to_show)
roi = ((x - window_size, x + window_size), (y - window_size, y + window_size))

In [ ]:
fig, ax = plt.subplots(figsize=(7*CM, 6*CM), dpi=600)
ax.set_facecolor("black")
img = ax.imshow(
        signal_integrity,
        cmap=BIH_CMAP,
        alpha=((signal_strength / signal_threshold).clip(0, 1) ** 2),
        vmin=0,
        vmax=1,
        rasterized=True
    )
ax.invert_yaxis()
ax.set(xlim=roi[0], ylim=roi[1])

cbar = fig.colorbar(img, ax=ax, shrink=0.95)
cbar.ax.tick_params(labelsize=6, width=0.3)
for spine in cbar.ax.spines.values():
    spine.set_linewidth(0.3)

_plot_scalebar(ax, dx=1, units="um", location="lower left", length_fraction=0.2, fontsize=5, color="white")

ax.set(xticks=[], yticks=[])
ax.set_title("Vertical Signal Integrity", fontsize=7)
ax.set_aspect('equal')

plt.show()

In [ ]:
roi_top = ROI_in.filter(pl.col("z") > pl.col("z_center"))

(x_min, x_max), (y_min, y_max) = roi
fig, ax = plt.subplots(figsize=(6*CM, 6*CM), dpi=600)
ax.scatter(
    roi_top["x"],
    roi_top["y"],
    c=roi_top["RGB"].to_numpy(),
    **roi_scatter_kwargs,
    edgecolors='none'
)
ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))

filtered_df = boundaries_df[
    (boundaries_df['x'] >= x_min) & (boundaries_df['x'] <= x_max) &
    (boundaries_df['y'] >= y_min) & (boundaries_df['y'] <= y_max)
]

ax.spines[["top", "right"]].set_visible(True)
for _, row in filtered_df.iterrows():
    ax.plot(row['boundaryX'], row['boundaryY'], c='#2C2C2C', linewidth=0.7)

ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect('equal')

for spine in ax.spines.values():
    spine.set_linewidth(0.3)

plt.title("Top", fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
roi_bottom = ROI_in.filter(pl.col("z") < pl.col("z_center"))[::-1]
(x_min, x_max), (y_min, y_max) = roi

fig, ax = plt.subplots(figsize=(6*CM, 6*CM), dpi=600)
ax.scatter(
    roi_bottom["x"],
    roi_bottom["y"],
    c=roi_bottom["RGB"].to_numpy(),
    **roi_scatter_kwargs,
    edgecolors='none',
)
ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))

filtered_df = boundaries_df[
    (boundaries_df['x'] >= x_min) & (boundaries_df['x'] <= x_max) &
    (boundaries_df['y'] >= y_min) & (boundaries_df['y'] <= y_max)
]

for _, row in filtered_df.iterrows():
    ax.plot(row['boundaryX'], row['boundaryY'], c='#2C2C2C', linewidth=0.7)

ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect('equal')

for spine in ax.spines.values():
    spine.set_linewidth(0.3)

plt.title("Bottom", fontsize=7)
plt.tight_layout()  
plt.show()

#### Doublet2: Excitatory

doublet_to_show = 158  

In [ ]:
ROI_ex =pl.read_parquet(ovrlpy_result_folder/"roi_transcripts_Ex.parquet")

doublet_to_show = 158
x, y = doublets["x", "y"].row(doublet_to_show)
roi = ((x - window_size, x + window_size), (y - window_size, y + window_size))

In [ ]:
fig, ax = plt.subplots(figsize=(7*CM, 6*CM), dpi=600)

ax.set_facecolor("black")
img = ax.imshow(
    signal_integrity,
    cmap=BIH_CMAP,
    alpha=((signal_strength / signal_threshold).clip(0, 1) ** 2),
    vmin=0,
    vmax=1
)
ax.invert_yaxis()
ax.set(xlim=roi[0], ylim=roi[1])

cbar = fig.colorbar(img, ax=ax, shrink=0.95)
cbar.ax.tick_params(labelsize=6, width=0.3)
for spine in cbar.ax.spines.values():
    spine.set_linewidth(0.3)

ax.set(xticks=[], yticks=[])
ax.set_title("Vertical Signal Integrity", fontsize=7)
ax.set_aspect('equal')

plt.show()

In [ ]:
roi_top = ROI_ex.filter(pl.col("z") > pl.col("z_center"))
(x_min, x_max), (y_min, y_max) = roi

fig, ax = plt.subplots(figsize=(6*CM, 6*CM), dpi=600)
ax.scatter(
    roi_top["x"],
    roi_top["y"],
    c=roi_top["RGB"].to_numpy(),
    **roi_scatter_kwargs,
    edgecolors='none'
)

filtered_df = boundaries_df[
    (boundaries_df['x'] >= x_min) & (boundaries_df['x'] <= x_max) &
    (boundaries_df['y'] >= y_min) & (boundaries_df['y'] <= y_max)
]

ax.spines[["top", "right"]].set_visible(True)
for _, row in filtered_df.iterrows():
    ax.plot(row['boundaryX'], row['boundaryY'], c='#2C2C2C', linewidth=0.7)

ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))
ax.set(xticks=[], yticks=[])
ax.set_aspect('equal')

for spine in ax.spines.values():
    spine.set_linewidth(0.3)

plt.title("Top", fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
roi_bottom = ROI_ex.filter(pl.col("z") < pl.col("z_center"))[::-1]
(x_min, x_max), (y_min, y_max) = roi

fig, ax = plt.subplots(figsize=(6*CM, 6*CM), dpi=600)
ax.scatter(
    roi_bottom["x"],
    roi_bottom["y"],
    c=roi_bottom["RGB"].to_numpy(),
    **roi_scatter_kwargs,
    edgecolors='none',
)

filtered_df = boundaries_df[
    (boundaries_df['x'] >= x_min) & (boundaries_df['x'] <= x_max) &
    (boundaries_df['y'] >= y_min) & (boundaries_df['y'] <= y_max)
]

ax.spines[["top", "right"]].set_visible(True)
for _, row in filtered_df.iterrows():
    ax.plot(row['boundaryX'], row['boundaryY'], c='#2C2C2C', linewidth=0.7)

ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))
ax.set(xticks=[], yticks=[])
ax.set_aspect('equal')

for spine in ax.spines.values():
    spine.set_linewidth(0.3)

plt.title("Bottom", fontsize=7)
plt.tight_layout()
plt.show()